In [2]:
# Importing the necessary modules
import pandas as pd
import tarfile
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [1]:
import os

In [2]:
print(os.getcwd())

/Users/tylerlewris/Documents/Technical Presentations


In [5]:
train_df = pd.read_csv('yelp_review_polarity_csv/train.csv', header=None)

train_df.head()

,0,1
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [9]:
test_df = pd.read_csv('yelp_review_polarity_csv/test.csv', header=None)

test_df.head()

,0,1
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...


In [10]:
train_df[0] = (train_df[0] == 2).astype(int)
test_df[0] = (test_df[0] == 2).astype(int)

In [15]:
train_df.head()

,0,1
0,0,"Unfortunately, the frustration of being Dr. Go..."
1,1,Been going to Dr. Goldberg for over 10 years. ...
2,0,I don't know what Dr. Goldberg was like before...
3,0,I'm writing this review to give you a heads up...
4,1,All the food is great here. But the best thing...


In [16]:
test_df.head()

,0,1
0,1,"Contrary to other reviews, I have zero complai..."
1,0,Last summer I had an appointment to get new ti...
2,1,"Friendly staff, same starbucks fair you get an..."
3,0,The food is good. Unfortunately the service is...
4,1,Even when we didn't have a car Filene's Baseme...


Making things BERT friendly
First let's make the data compliant with BERT:

- Column 0: An ID for the row. (Required both for train and test data.)
- Column 1: The class label for the row. (Required only for train data.)
- Column 2: A column of the same letter for all rows — this is a throw-away column that we need to include because   BERT expects it. (Required only for train data.)
- Column 3: The text examples we want to classify. (Required both for train and test data.)

We need to split the files into the format expected by BERT: BERT comes with data loading classes that expects two files called train and dev for training. In addition, BERT’s data loading classes can also use a test file but it expects the test file to be unlabelled.


Once the data is in the correct format, we need to save the files as .tsv (BERT doesn't take .csv as input.)

In [17]:
# Creating training dataframe according to BERT by adding the required columns
df_bert = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_df[0],
    'alpha':['a']*train_df.shape[0],
    'text': train_df[1].replace(r'\n', ' ', regex=True)
})


# Splitting training data file into *train* and *dev*
df_bert_train, df_bert_dev = train_test_split(df_bert, test_size=0.01)

df_bert_train.head()

,id,label,alpha,text
122425,122425,0,a,I'm giving 2 stars because the wait staff was ...
118172,118172,1,a,Our supplier invited some of their loyal clien...
212082,212082,0,a,I tried this restaurant once. They used so mu...
3502,3502,0,a,use to be a max and erma's and if that bar was...
82953,82953,1,a,Let me just start by saying that i DO personal...


In [19]:
df_bert_train['text'][0]

"Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patients with medical needs, why isn't anyone answering the phone?  It's incomprehensible and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 stars."

In [20]:
# Creating test dataframe according to BERT
df_bert_test = pd.DataFrame({
    'id':range(len(test_df)),
    'text': test_df[1].replace(r'\n', ' ', regex=True)
})

df_bert_test.head()

,id,text
0,0,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,3,The food is good. Unfortunately the service is...
4,4,Even when we didn't have a car Filene's Baseme...


In [25]:
df_bert_dev.shape

(5600, 4)

In [26]:
# Saving dataframes to .tsv format as required by BERT
df_bert_train.to_csv('train.tsv', sep='\t', index=False, header=False)
df_bert_dev.to_csv('dev.tsv', sep='\t', index=False, header=False)
df_bert_test.to_csv('test.tsv', sep='\t', index=False, header=False)

in bert optimization file you need to replace line 87 with tf.keras.optimizers.Optimizer